In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from pathlib import Path
from collections import Counter

In [2]:
# Loading the Dataset
file_path = Path('./Resources/clean_tvfilm.csv')
tvfilm_df = pd.read_csv(file_path)
tvfilm_df.head()

,filmtv_id,title,year,genre,duration,country,directors,actors,avg_vote,critics_vote,public_vote,total_votes,description,notes,humor,rhythm,effort,tension,erotism
0,3,18 anni tra una settimana,1991,Drama,98,Italy,Luigi Perelli,"Kim Rossi Stuart, Simona Cavallari, Ennio Fant...",6.5,6.00,7,4,"Samantha, not yet eighteen, leaves the comfort...","Luigi Perelli, the director of the ""Piovra"", o...",0,2,0,2,0
1,17,Ride a Wild Pony,1976,Romantic,91,United States,Don Chaffey,"Michael Craig, John Meillon, Eva Griffith, Gra...",5.6,6.00,5,9,"In the Australia of the pioneers, a boy and a ...","""Ecological"" story with a happy ending, not wi...",1,2,1,0,0
2,18,Diner,1982,Comedy,95,United States,Barry Levinson,"Mickey Rourke, Steve Guttenberg, Ellen Barkin,...",7.0,8.00,6,18,Five boys from Baltimore have a habit of meeti...,A cast of will be famous for Levinson's direct...,2,2,0,1,2
3,20,A che servono questi quattrini?,1942,Comedy,85,Italy,Esodo Pratelli,"Eduardo De Filippo, Peppino De Filippo, Clelia...",5.9,5.33,7,15,"With a stratagem, the penniless and somewhat p...",Taken from the play by Armando Curcio that the...,3,1,1,0,0
4,21,The Uranian Conspiracy,1978,Spy,117,"Italy, Germany, Israel","Gianfranco Baldanello, Menahem Golan","Fabio Testi, Janet Agren, Assaf Dayan, Siegfri...",4.8,3.50,6,3,Two Israeli secret agents discover that traffi...,"Action and chases for half of Europe, espionag...",1,2,0,2,0


In [3]:
# Creating a threshold for 'avg_vote' column
threshold = 3

# Creating a new column determining the films success
tvfilm_df['is_successful'] = tvfilm_df['humor'].apply(lambda x: 'is_successfil' 
                                                         if x >= threshold else 
                                                         'not_successful')
# Splitting the data into training and testing
X = tvfilm_df[['humor', 'rhythm', 'effort', 'tension', 'erotism']]
y = tvfilm_df['is_successful']

In [4]:
# Select the features (avg_vote, duration) and target (is_successful)
X = tvfilm_df[['humor', 'rhythm', 'effort', 'tension', 'erotism']]
y = tvfilm_df['is_successful']

In [5]:
X.describe()

,humor,rhythm,effort,tension,erotism
count,17073.000000,17073.00000,17073.000000,17073.000000,17073.000000
mean,0.891583,2.02501,1.037428,1.373279,0.482692
std,0.981624,0.82701,1.233634,1.097787,0.742706
min,0.000000,0.00000,0.000000,0.000000,0.000000
25%,0.000000,2.00000,0.000000,0.000000,0.000000
50%,1.000000,2.00000,1.000000,1.000000,0.000000
75%,2.000000,2.00000,2.000000,2.000000,1.000000
max,5.000000,5.00000,5.000000,5.000000,4.000000


In [6]:
# Check the balance of our target values
y.value_counts()

not_successful    15893
is_successfil      1180
Name: is_successful, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(12804, 5)

In [8]:
### OVERSAMPLING
## Native Random Oversampling
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'not_successful': 11919, 'is_successfil': 11919})

In [9]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [10]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

1.0

In [11]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 295,    0],
       [   0, 3974]])

In [12]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                      pre       rec       spe        f1       geo       iba       sup

 is_successfil       1.00      1.00      1.00      1.00      1.00      1.00       295
not_successful       1.00      1.00      1.00      1.00      1.00      1.00      3974

   avg / total       1.00      1.00      1.00      1.00      1.00      1.00      4269



In [13]:
### OVERSAMPLING
## SMOTE Oversampleing
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(
    X_train, y_train
)
Counter(y_resampled)

Counter({'not_successful': 11919, 'is_successfil': 11919})

In [14]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [15]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

1.0

In [16]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[ 295,    0],
       [   0, 3974]])

In [17]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                      pre       rec       spe        f1       geo       iba       sup

 is_successfil       1.00      1.00      1.00      1.00      1.00      1.00       295
not_successful       1.00      1.00      1.00      1.00      1.00      1.00      3974

   avg / total       1.00      1.00      1.00      1.00      1.00      1.00      4269



In [18]:
### UNDERSAMPLING
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

/Users/kweeniehutjrs/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/imblearn/under_sampling/_prototype_generation/_cluster_centroids.py:178: ConvergenceWarning: Number of distinct clusters (784) found smaller than n_clusters (885). Possibly due to duplicate points in X.
  self.estimator_.fit(_safe_indexing(X, target_class_indices))


Counter({'is_successfil': 885, 'not_successful': 885})

In [19]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [20]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

1.0

In [21]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[ 295,    0],
       [   0, 3974]])

In [22]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                      pre       rec       spe        f1       geo       iba       sup

 is_successfil       1.00      1.00      1.00      1.00      1.00      1.00       295
not_successful       1.00      1.00      1.00      1.00      1.00      1.00      3974

   avg / total       1.00      1.00      1.00      1.00      1.00      1.00      4269



In [23]:
### COMBINATION OF (OVER AND UNDER) SAMPBLING
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

Counter({'is_successfil': 15893, 'not_successful': 15892})

In [24]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [25]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

1.0

In [26]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 295,    0],
       [   0, 3974]])

In [27]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                      pre       rec       spe        f1       geo       iba       sup

 is_successfil       1.00      1.00      1.00      1.00      1.00      1.00       295
not_successful       1.00      1.00      1.00      1.00      1.00      1.00      3974

   avg / total       1.00      1.00      1.00      1.00      1.00      1.00      4269

